# OpenFMRIFlanker normalize data

This notebook pulls data from fmriprep and nibetaseries to do some preprocessing

In [46]:
# grabs files based on globbing
from glob import glob
# load nifti files as a python object
import nibabel as nib
# represent data as arrays (faster manipulation and smarter indexing)
import numpy as np
# complete basic statistical operations on numpy arrays
from scipy import stats

In [161]:
# the mask for all files
mask = '../group/group_bold_brainmask.nii.gz'
# the betaseries files
files = glob('../sub-*/func/sub-*_trialtype-*_betaseries.nii.gz')

### zscoring commands
the below commands load in the nifti images and perform zscoring along the 4th dimension.

In [162]:
for file in files:
    # load nifti
    orig_img = nib.load(file)

    # load img object into numpy array
    orig_data_4d = orig_img.get_data()

    # get the number of volumes (assuming 4d)
    num_vols = orig_data_4d.shape[-1]

    # load nifti
    mask_img = nib.load(mask)

    # load img into 3d numpy array
    mask_data = mask_img.get_data()

    # treat the array as boolean (T & F)
    mask_data = mask_data.astype(bool)

    # mask the data to only indices we want
    masked_data_2d = orig_data_4d[mask_data]

    # zscore the masked data across time (not space)
    zscore_data_2d = stats.zscore(masked_data_2d, axis=1)

    # set up output array
    zscore_data_4d = np.zeros(orig_data_4d.shape, dtype=orig_data_4d.dtype)

    # set the output data
    zscore_data_4d[mask_data] = zscore_data_2d

    # set headers from original image
    img_header = orig_img.header
    img_affine = orig_img.affine

    # zscored nibabel img
    zscore_img = nib.Nifti1Image(zscore_data_4d, img_affine, img_header)

    # save file to correct place
    zscore_img.to_filename(file.replace('betaseries','zscore'))